In [90]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import obspy
from obspy.signal import PPSD
from obspy.imaging.cm import pqlx
import pandas as pd
from datetime import datetime

In [86]:
def calculate_dB(axis):
    axis = np.array(axis)
    axis = axis - np.mean(axis)
    axis = axis * 9.81 # G to m/s^2
    rms = np.sqrt(np.mean(axis ** 2)) # rms
    dB = 20 * np.log10(rms)
    return dB

def plot_line(data, axis='x'):
    min_, max_ = np.min(data), np.max(data)
    plt.ylim([min_ - 0.005, max_ + 0.005])
    plt.plot(data)
    plt.title(axis + '-axis')
    plt.show()
    
def plot_scatter(data, ts, axis='x'):
    min_, max_ = np.min(data), np.max(data)
    plt.ylim([min_ - 0.005, max_ + 0.005])
    plt.scatter(ts, data)
    plt.title(axis + '-axis')
    plt.show()

In [91]:
def get_trace(dev, data, channel, stime, sampling_rate=100):
    tr = obspy.Trace()
    tr.data = data
    tr.stats['sampling_rate'] = sampling_rate
    tr.stats['npts'] = len(data)
    tr.stats['channel'] = channel
    tr.stats['station'] = dev
    tr.stats.starttime = stime
    
    return tr

def get_stream(dev, xdata, ydata, zdata, stime, sampling_rate=100):
    st = obspy.Stream()
    st.append(get_trace(dev, xdata.to_numpy(), 'x', stime, sampling_rate))
    st.append(get_trace(dev, ydata.to_numpy(), 'y', stime, sampling_rate))
    st.append(get_trace(dev, zdata.to_numpy(), 'z', stime, sampling_rate))
    return st

def get_psd(st, channel, paz, outpath='ppsd.png'):  
    tr = st.select(channel=channel)[0]
    ppsd = PPSD(tr.stats, paz, ppsd_length=1800.0, db_bins=(-200, 0, 1.), special_handling='ringlaser')
    ppsd.add(st)
    ppsd.plot(filename=outpath, cmap=pqlx)
    psd = {}
    for utc in ppsd.times_processed:
        psd[str(utc)] = ppsd.extract_psd_values(float(utc))
    return psd

In [109]:

Path('output').mkdir(exist_ok=True, parents=True)

csv_path = 'output/analysis.csv'
cache_path = 'cache'
columns = 'device,stime,psd_x,psd_y,psd_z,cnt\n'

if not Path(csv_path).exists():
    with open(csv_path, 'w+') as f:
        f.write(columns)

strange = []
sampling_rate = 100
cache = []

if not Path(cache_path).exists():
    with open(cache_path, 'w+') as f:
        pass
    #Path(cache_path).touch(mode=755)
    #Path(cache_path).chmod(755)
else:
    with open(cache_path, 'r') as f:
        print(f.readlines())

def analyze(device, file):
    if file in cache:
        print(f'Already analyzed file: {file}')
        return
    
    img_out = Path(f'./output/img/{device}/')
    img_out.mkdir(exist_ok=True, parents=True)
    
    csv_out = Path(f'./output/')
    csv_out.mkdir(exist_ok=True, parents=True)
    
    print(f'Reading {file}')
    
    df = pd.read_csv(
            str(file),
            names=['x', 'y', 'z', 'ts'],
            header=None
        )
    
    if df.shape[0] == 0:
        print(f'No data in this file: failed: {file}')
        return
    
    stime = datetime.fromtimestamp(df.iloc[0]['ts'])
    
    st = get_stream(device, df['x'], df['y'], df['z'], stime, sampling_rate)
    st.plot(outfile=str(img_out / f'{str(stime)}_noise.png'), equal_scale=False)
    
    try:
        psds = [
            get_psd(st, 'x', {'sensitivity': 1}, str(img_out / f'{stime}_ppsd_x.png')),
            get_psd(st, 'y', {'sensitivity': 1}, str(img_out / f'{stime}_ppsd_y.png')),
            get_psd(st, 'z', {'sensitivity': 1}, str(img_out / f'{stime}_ppsd_z.png')),
        ]
    except Exception as e:
        # starnge string in file
        print(f'Strange value in this file: Failed: {file}')
        print(e)
        strange.append(file)
        return

    psd_values = []
    for _, psd in enumerate(psds):
        values = []
        for key in psd:
            values.append(psd[key][0])
        psd_values.append(np.mean(values))
    
    with open(csv_path, 'a+') as f:
        f.write(f'{device},{stime},{psd_values[0]},{psd_values[1]},{psd_values[2]}\n')
        
    print(f'Success: {file}')
    with open(cache_path, 'a+') as f:
        f.write(f'{file}\n')
        cache.append(file)
    

In [111]:
for file in Path('data').rglob('*.csv'):
    device = file.relative_to('data').parent
    analyze(device, str(file))

Already analyzed file: data/shake1/2022-03-10T09:29:35.113616.csv
Already analyzed file: data/shake2/2022-03-10T09:24:12.552938.csv
